In [1]:
import torch
import numpy as np
import pandas as pd
import re
import torch
import sklearn
import nltk
import joblib
import pandas as pd

from transformers import AutoTokenizer, T5ForConditionalGeneration
from transformers import set_seed

import clickhouse_connect

from sentence_transformers import SentenceTransformer
from FlagEmbedding import BGEM3FlagModel

from sklearn.neighbors import NearestNeighbors

from utils.utils import Chunker, Generator, Retriever
from utils.llm_config import GenerationConfig

device = 'cuda'

set_seed(42)

nltk.download('punkt')
torch.cuda.is_available()

/home/jupyter/.local/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
2024-03-13 22:04:45.300866: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[nltk_data] Downloading package punkt to /home/jupyter/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
client = clickhouse_connect.get_client(host='62.84.115.43', port=8123, username='default', password='g5oHPYX4', database='dev')

## Подгрузка моделей

In [3]:
bge_retriever_model = BGEM3FlagModel('BAAI/bge-m3', use_fp16=True, pooling_method="cls")
e5_retriever_model = SentenceTransformer("intfloat/multilingual-e5-large", device='cuda')
e5_retriever_model.max_seq_length = 512

Fetching 23 files:   0%|          | 0/23 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

imgs/.DS_Store:   0%|          | 0.00/6.15k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

...

loading existing colbert_linear and sparse_linear---------


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

In [4]:
generator_model_name = 'hivaze/AAQG-QA-QG-FRED-T5-1.7B'
generator_tokenizer = AutoTokenizer.from_pretrained(generator_model_name)
generator_model = T5ForConditionalGeneration.from_pretrained(generator_model_name).cuda().eval()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Chunker

In [12]:
chunker = Chunker(max_chunk_len=2500, overlap_len=500)


texts = client.query_df('select * from document').set_index('url')
chunks, urls = chunker.split_texts(texts)
client.insert(table='chunk', data=[chunks, urls], column_names=['text', 'url'], column_oriented=True)

In [8]:
# encoded_queries = bge_retriever_model.encode(book["query"].tolist(), return_dense=True, return_sparse=False, return_colbert_vecs=True, batch_size=16, max_length=512)

In [6]:
chunks_df = client.query_df('select * from chunk').set_index('uuid')

In [8]:
bge_embeddings = bge_retriever_model.encode(chunks_df.text.tolist(), return_dense=True, return_sparse=False, return_colbert_vecs=False, batch_size=16, max_length=512)
e5_embeddings = e5_retriever_model.encode(chunks_df.text.tolist(), batch_size=16, normalize_embeddings=True, show_progress_bar=True)
client.insert(table='chunk_embedding',
              data=[chunks_df.index.tolist(), 
                    bge_embeddings['dense_vecs'].tolist(), 
                    e5_embeddings.tolist()],
              column_names=['chunk_uuid', 
                            'bge_m3_embedding', 
                            'e5_embedding'], column_oriented=True)

Inference Embeddings: 100%|██████████| 2705/2705 [03:17<00:00, 13.70it/s]


Batches:   0%|          | 0/2705 [00:00<?, ?it/s]

In [9]:
client.query_df('select * from chunk_embedding')

,chunk_uuid,bge_m3_embedding,e5_embedding
0,09785425-2e5a-4450-8000-25898e9ee517,"[-0.017211914, 0.011268616, -0.002105713, 0.03...","[-0.00078974775, -0.004833441, -0.010927365, -..."
1,5a190169-dac3-496b-8001-4b2e0e0d6fc5,"[0.012702942, -0.0025024414, -0.020629883, 0.0...","[0.023522928, -0.007914214, -0.035638273, -0.0..."
2,4f94df03-6da5-4b0d-8002-1d3cbb2134dd,"[-0.013771057, -0.019805908, -0.015159607, 0.0...","[-0.0012714802, -0.0009910531, -0.017019093, -..."
3,9b010e2e-42b3-4eb1-8002-3017aa2a4775,"[-0.010108948, 0.018081665, -0.030929565, 0.00...","[0.024700968, -0.029986667, -0.043692064, -0.0..."
4,4f00e2a8-4aed-49c9-8002-4a7588770c40,"[-0.021575928, 0.0028858185, -0.039886475, -0....","[0.023082534, -0.02455546, -0.013776501, -0.04..."
...,...,...,...
1306,c9990629-5009-40de-b458-ebd414320e9e,"[-0.006462097, 0.003326416, -0.016204834, 0.03...","[-0.006397617, -0.027702993, -0.014129917, -0...."
1307,c4515a46-cca4-43b1-b459-035699a46439,"[-0.024795532, -0.021865845, -0.0020008087, -0...","[0.006878287, -0.023127899, -0.0028097988, -0...."
1308,1c3a94de-8506-4f4d-b459-24cdcbdd68ba,"[-0.0076522827, -0.026367188, -0.037719727, -0...","[-0.004693624, -0.017884552, -0.022393245, -0...."
1309,b8e68c20-2a91-4c53-b459-e3a275c1a5f0,"[0.016235352, 0.022705078, 0.0056762695, -0.01...","[0.01902052, -0.0034569488, -0.024087856, -0.0..."


## Retriever & Generator

In [15]:
generator_config = GenerationConfig(num_beams=3)
generator = Generator(generator_tokenizer, generator_model, config=generator_config)
retriever = Retriever(e5_retriever_model, bge_retriever_model, client)

In [6]:
queries = pd.read_csv('Book1 (1).csv', sep=';')
queries.site_id = queries.site_id.apply(lambda x: x if  x[-1] == '/' else x+'/')
queries.site_id = queries.site_id.apply(lambda x: x if 'http://pravo.gov.ru' in x else 'https://cbr.ru' + x)

In [17]:
question = queries.question.sample(1).item()
question

'Кто может принимать участие в закупках Банка России?'

In [18]:
topk_texts, topk_urls, question_embedding = retriever.get_neighbors(question)
rerank_indeces = retriever.rerank(question_embedding, topk_texts, 3)
reranked_texts, reranked_urls = retriever.get_topk(topk_texts, topk_urls, rerank_indeces)

In [23]:
print(generator.get_answer(question, reranked_texts, reranked_urls, temperature=0.6))

Участником закупки Банка России может стать любое юридическое лицо независимо от его организационно -правовой формы, формы собственности, места нахождения и места происхождения капитала.

Использованные документы:
1) https://cbr.ru/Crosscut/LawActs/File/6620/ 

